# Lab02: Frequent itemset mining

- Student ID: 21127627
- Student name: Cao Nguyễn Khánh

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = a.union(b)
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below? 

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        
        # Duyệt qua mỗi transaction trong dữ liệu
        for transaction in self.data.values():
            # Nếu itemset là một phần của transaction, tăng giá trị của support
            if set(itemset).issubset(transaction):
                support += 1
        
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        for item, item_info in search_space.items():
            # print(item, item_info)
            if item_info['support'] >= self.minSup:
                self.L[k].append(item)
            else:
                item_info['pruned'] = True
                del tree[item]



    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # return
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(set(search_space[a]['itemset']), set(search_space[b]['itemset']))
                
                
                # Second add newset to search_space
                newset = tuple(newset)
                sub_search_space[newset] = {'itemset': list(newset), 'pruned': False, 'support': self.computeItemsetSupport(newset)}

                # print(search_space[newset])
                sub_tree[newset] = {}

     
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        # print(search_space)
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')



In [5]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [48, 56, 66, 34, 62, 7, 36, 60, 40, 29, 52, 58], 2: [(48, 52), (48, 58), (56, 29), (56, 52), (56, 58), (66, 60), (66, 29), (66, 52), (66, 58), (40, 34), (34, 29), (34, 52), (34, 58), (60, 62), (40, 62), (29, 62), (52, 62), (58, 62), (60, 7), (40, 7), (29, 7), (52, 7), (58, 7), (36, 60), (40, 36), (36, 29), (36, 52), (58, 36), (40, 60), (60, 29), (60, 52), (58, 60), (40, 29), (40, 52), (40, 58), (52, 29), (58, 29), (58, 52)], 3: [(48, 58, 52), (56, 52, 29), (56, 58, 29), (56, 58, 52), (66, 60, 29), (66, 60, 52), (66, 60, 58), (66, 52, 29), (66, 58, 29), (66, 52, 58), (40, 34, 29), (40, 34, 52), (40, 34, 58), (34, 52, 29), (34, 58, 29), (34, 52, 58), (60, 29, 62), (60, 62, 52), (58, 60, 62), (40, 29, 62), (40, 52, 62), (40, 58, 62), (52, 29, 62), (58, 29, 62), (58, 52, 62), (40, 60, 7), (60, 29, 7), (60, 52, 7), (58, 60, 7), (40, 29, 7), (40, 52, 7), (40, 58, 7), (52, 29, 7), (58, 29, 7), (58, 52, 7), (40, 36, 60), (36, 29, 60), (36, 60, 52), (58, 36, 60), (40, 36, 29), (40, 36, 52),

### Answer questions here:
**Why don't we compute support of items while reading data?**


**why should we do sort**
- Vì dòng code sau : 
```
    for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                HERE
```

- Khi dùng sorted thì ở vòng lặp thứ 2 chúng ta sẽ không cần kiểm tra điều kiện của *['pruned']* của items thứ j+1. Bì nếu a lớn hơn minSup thì chắc chắn những Items sau a cũng sẽ lớn hơn minSup. 
- Giả sử nếu không sorted thì cứ mỗi lần có một phần tử vượt qua kiểm tra điều kiện của *['pruned']* thì phải duyệt và kiểm tra hết toàn bộ những items sau đó. 

**study about python set and its advantages ?**
- Set trong Python đảm bảo rằng mỗi phần tử chỉ xuất hiện duy nhất một lần, Không bị lỗi đếm 1 phần tử 2 lần.
- Việc kiểm tra xem một set con có là một phần của một set lớn hơn có thể được thực hiện bằng cách sử dụng phép toán issubset(). Điều này giúp giảm thiểu độ phức tạp của thuật toán.
- Các phép toán trên set trong Python thường có thời gian chạy nhanh hơn so với các cấu trúc dữ liệu khác như danh sách (list) hoặc từ điển (dictionary).

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**
- vì thuật toán sử dụng đệ quy, khi xóa cấu trúc dữ liệu sẽ làm ảnh hưởng đến thuật toán khi ta quay lui.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**
- Apriori : Apriori chỉ có thể tạo ra các Items ứng viên với kích thước k + 1. Với kiểu kiện 2 Items mẹ (kích thước k) phải có k-1 phần tử đầu giống nhau.
- Tree Projection: Tree Projection có thể tạo ra các Items ứng viên mà không cần quan tâm diều kiện kích thước tập sau là k + 1.

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [6]:
# TODO
# YOUR CODE HERE

import pandas as pd
# Đường dẫn đến file txt
file_path = "churn.txt"

# Đọc dữ liệu từ file txt và chuyển sang DataFrame
df = pd.read_csv(file_path)




In [7]:
# Hiển thị DataFrame
print(df.columns)

df = df.drop(['Area Code', 'Phone'], axis=1)

# df = df['State']

df.info()


Index(['State', 'Account Length', 'Area Code', 'Phone', 'Int'l Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn?'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Int'l Plan      3333 non-null   object 
 3   VMail Plan      3333 non-null   object 
 4   VMail Message   3333 non-null   int64  
 5   Day Mins        3333 non-null   float64
 6   Day Calls       3333 non-null   int64  
 7   Day Charge      3333 non-null   float64
 8   Eve Mins        3333 non-null   float64
 9   Eve Calls       3333 non-null   int64  
 10  Eve

In [8]:
# Lấy các cột có kiểu dữ liệu là 'int64' hoặc 'float64'
numeric_cols = df.select_dtypes(include=['int64', 'float64'])

# Tính phân vị của mỗi cột
percentiles = numeric_cols.quantile([0.33, 0.66])

# Hàm ánh xạ giá trị sang nhóm cao, trung bình, thấp
def map_to_group(x, col_percentiles):
    if x <= col_percentiles[0.33]:
        return 'Low'
    elif x <= col_percentiles[0.66]:
        return 'Medium'
    else:
        return 'High'

# Áp dụng hàm map_to_group cho từng cột
for col in numeric_cols.columns:
    numeric_cols[col] = numeric_cols[col].apply(map_to_group, col_percentiles=percentiles[col])

numeric_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Account Length  3333 non-null   object
 1   VMail Message   3333 non-null   object
 2   Day Mins        3333 non-null   object
 3   Day Calls       3333 non-null   object
 4   Day Charge      3333 non-null   object
 5   Eve Mins        3333 non-null   object
 6   Eve Calls       3333 non-null   object
 7   Eve Charge      3333 non-null   object
 8   Night Mins      3333 non-null   object
 9   Night Calls     3333 non-null   object
 10  Night Charge    3333 non-null   object
 11  Intl Mins       3333 non-null   object
 12  Intl Calls      3333 non-null   object
 13  Intl Charge     3333 non-null   object
 14  CustServ Calls  3333 non-null   object
dtypes: object(15)
memory usage: 390.7+ KB


In [9]:
# Lấy các cột không phải là 'int64' hoặc 'float64'
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64'])
non_numeric_cols

# Hợp lại các DataFrame non_numeric_cols và numeric_cols
merged_df = pd.concat([non_numeric_cols, numeric_cols], axis=1)

# Hiển thị DataFrame đã hợp lại
print("DataFrame sau khi hợp lại:")
merged_df

DataFrame sau khi hợp lại:


,State,Int'l Plan,VMail Plan,Churn?,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
0,KS,no,yes,False.,High,High,High,High,High,Medium,Medium,Medium,High,Low,High,Medium,Low,Medium,Low
1,OH,no,yes,False.,Medium,High,Medium,High,Medium,Medium,Medium,Medium,High,Medium,High,High,Low,High,Low
2,NJ,no,no,False.,High,Low,High,High,High,Low,High,Low,Low,Medium,Low,High,Medium,High,Low
3,OH,yes,no,False.,Low,Low,High,Low,High,Low,Low,Low,Medium,Low,Medium,Low,High,Low,Medium
4,OK,yes,no,False.,Low,Low,Medium,High,Medium,Low,High,Low,Medium,High,Medium,Medium,Low,Medium,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,no,yes,False.,High,High,Medium,Low,Medium,Medium,High,Medium,High,Low,High,Medium,High,Medium,Medium
3329,WV,no,no,False.,Low,Low,High,Low,High,Low,Low,Low,Medium,High,Medium,Medium,Medium,Medium,High
3330,RI,no,no,False.,Low,Low,Medium,Medium,Medium,High,Low,High,Medium,Low,Medium,High,High,High,Medium
3331,CT,yes,no,False.,High,Low,High,Medium,High,Low,Low,Low,Low,High,Low,Low,High,Low,Medium


In [10]:
# Áp dụng mã hóa one-hot cho DataFrame
encoded_df = pd.get_dummies(merged_df)
encoded_df


,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,...,Intl Mins_Medium,Intl Calls_High,Intl Calls_Low,Intl Calls_Medium,Intl Charge_High,Intl Charge_Low,Intl Charge_Medium,CustServ Calls_High,CustServ Calls_Low,CustServ Calls_Medium
0,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,True,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,False,False,False,True,False,False,False,False,False,False,...,True,True,False,False,False,False,True,False,False,True
3329,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,True,False,False
3330,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,True
3331,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,False,True


In [11]:

def df_to_dic(df):

    # Tạo một từ điển để lưu trữ kết quả
    result_dict = {}
    s=defaultdict(lambda: 0)
    
    # Lặp qua mỗi hàng của DataFrame
    for index, row in df.iterrows():
        # Tạo một từ điển để lưu trữ tên các cột có giá trị True trong hàng hiện tại
        true_columns = []
        
        # Lặp qua mỗi phần tử trong hàng
        for column, value in row.items():
            # Kiểm tra nếu giá trị là True
            if value == True:
                # Thêm tên cột vào từ điển
                # column = int(column)
                true_columns.append(column)
                s[column] +=1 
    
        true_columns = set(true_columns)
        
        # Lưu kết quả vào từ điển với key là thứ tự dòng và giá trị là list các tên cột
        result_dict[index+1] = true_columns
        
        
    return result_dict, s

result_dict, s_churn = df_to_dic(encoded_df)

In [12]:
a=TP(data=result_dict,s=s_churn, minSup=1500)
print(a.miningResults())

{1: ['CustServ Calls_Low', 'VMail Plan_no', 'VMail Message_Low', 'Churn?_False.', "Int'l Plan_no"], 2: [('Churn?_False.', 'CustServ Calls_Low'), ("Int'l Plan_no", 'CustServ Calls_Low'), ('VMail Plan_no', 'VMail Message_Low'), ('VMail Plan_no', 'Churn?_False.'), ('VMail Plan_no', "Int'l Plan_no"), ('VMail Message_Low', 'Churn?_False.'), ("Int'l Plan_no", 'VMail Message_Low'), ("Int'l Plan_no", 'Churn?_False.')], 3: [("Int'l Plan_no", 'Churn?_False.', 'CustServ Calls_Low'), ('VMail Plan_no', 'VMail Message_Low', 'Churn?_False.'), ('VMail Plan_no', "Int'l Plan_no", 'VMail Message_Low'), ('VMail Plan_no', "Int'l Plan_no", 'Churn?_False.'), ("Int'l Plan_no", 'VMail Message_Low', 'Churn?_False.')], 4: [('VMail Plan_no', "Int'l Plan_no", 'VMail Message_Low', 'Churn?_False.')]}
